In [ ]:
#############################################################################
### This script computes various network metrics from CSV files containing
### weighted adjacency matrices with coordinates.
### DEBUGGED AND EDITED
#############################################################################

In [9]:
import numpy as np
import networkx as nx
import os, glob
import pandas as pd
from scipy import stats
from  scipy.special import rel_entr
import matplotlib.pyplot as plt
from networkx.algorithms import community
import seaborn as sns
from pathlib import Path

In [ ]:
#############################################################################################
# Define the directory and file patterns
# Adjust the paths as necessary for your environment
#############################################################################################

In [ ]:
#############################################################################################
### Simulations with High Initial Soil Fertility (Index 8400) and 40% Initial Reposition (adjustable)
#############################################################################################
import os, glob
import pandas as pd
data_dir = r"C:\Users\myPC\Agroforestal\Discover Networks\Simulations\Simulation6"
file_path = r"C:\Users\myPC\Agroforestal\Discover Networks\Simulations\Simulation6\weighted_matrix_with_coords_combined_tick_182.csv"

# Collect all files matching the pattern

pattern      = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
harvest_file = os.path.join(data_dir, "harvest_per_plant_40RepositionVariable.csv")
harvest_df   = pd.read_csv(harvest_file)

# Clean up: convert 'tick' and 'harvested' to numeric, drop rows with NaN
harvest_df['tick'] = pd.to_numeric(harvest_df['tick'], errors='coerce')
harvest_df['harvested'] = pd.to_numeric(harvest_df['harvested'], errors='coerce')
harvest_df = harvest_df.dropna(subset=['tick', 'harvested'])

# make a dict:  tick → np.array of harvested values (length 400 each tick)
harvest_by_tick = {
    int(t): grp["harvested"].to_numpy(float)
    for t, grp in harvest_df.groupby("tick")
}

output_dir = os.path.join(data_dir, "network_plots")
os.makedirs(output_dir, exist_ok=True)
save_dir     = output_dir

files = sorted(glob.glob(pattern),          # FIX glob call
               key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))


In [ ]:
#############################################################################################
### Simulations with High Initial Soil Fertility (Index 8400) and 45% Initial Reposition (fixed)
#############################################################################################
data_dir = r"C:\Users\myPC\Agroforestal\Discover Networks\Simulations\Simulation5"
file_path = r"C:\Users\myPC\Agroforestal\Discover Networks\Simulations\Simulation5\weighted_matrix_with_coords_combined_tick_182.csv"

# Collect all files matching the pattern

pattern      = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
harvest_file = os.path.join(data_dir, "harvest_per_plant_45FixedReposition.csv")
harvest_df   = pd.read_csv(harvest_file)

# Clean up: convert 'tick' and 'harvested' to numeric, drop rows with NaN
harvest_df['tick'] = pd.to_numeric(harvest_df['tick'], errors='coerce')
harvest_df['harvested'] = pd.to_numeric(harvest_df['harvested'], errors='coerce')
harvest_df = harvest_df.dropna(subset=['tick', 'harvested'])

# make a dict:  tick → np.array of harvested values (length 400 each tick)
harvest_by_tick = {
    int(t): grp["harvested"].to_numpy(float)
    for t, grp in harvest_df.groupby("tick")
}

output_dir = os.path.join(data_dir, "network_plots")
os.makedirs(output_dir, exist_ok=True)

files = sorted(glob.glob(pattern),          # FIX glob call
               key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))


In [ ]:
#############################################################################################
### Simulations with Low Initial Soil Fertility (Index 4400) and 50% Initial Reposition (adjustable)
#############################################################################################
data_dir = r"C:\Users\myPC\Agroforestal\Discover Networks\Simulations\Simulation7"
file_path = r"C:\Users\myPC\Agroforestal\Discover Networks\Simulations\Simulation7\weighted_matrix_with_coords_combined_tick_182.csv"

# Collect all files matching the pattern

pattern      = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
harvest_file = os.path.join(data_dir, "harvest_per_plant_50RepositionVariable.csv")
harvest_df   = pd.read_csv(harvest_file)

# Clean up: convert 'tick' and 'harvested' to numeric, drop rows with NaN
harvest_df['tick'] = pd.to_numeric(harvest_df['tick'], errors='coerce')
harvest_df['harvested'] = pd.to_numeric(harvest_df['harvested'], errors='coerce')
harvest_df = harvest_df.dropna(subset=['tick', 'harvested'])

# make a dict:  tick → np.array of harvested values (length 400 each tick)
harvest_by_tick = {
    int(t): grp["harvested"].to_numpy(float)
    for t, grp in harvest_df.groupby("tick")
}

output_dir = os.path.join(data_dir, "network_plots")
os.makedirs(output_dir, exist_ok=True)

files = sorted(glob.glob(pattern),          # FIX glob call
               key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))


In [ ]:
#############################################################################################
# Define the target ticks for analysis and define dictionaries for network, harvest, and soil files
#############################################################################################

target_ticks = [128, 180, 336, 388, 440, 492]
# target_ticks = [24, 76, 128, 180, 232, 284, 336, 388, 440, 492]

for tick in target_ticks:
    print(f"Preparing files for tick {tick}")
    
    # Create dictionaries for each tick
    network_files = {tick: f"{data_dir}/weighted_matrix_with_coords_combined_tick_{tick + 2}.csv"}
    harvest_file = os.path.join(data_dir, "harvest_per_plant_40RepositionVariable.csv")
    soil_file = f"{data_dir}/comprehensive_soil_data_40Variable.csv"

    weights_file = fr"{data_dir}\weighted_matrix_xy_combined_tick_{tick+ 2}.csv"
    W = pd.read_csv(weights_file, index_col=0).to_numpy(float)

    # flatten all finite weights, keep negatives too
    weights_flat = W[np.isfinite(W)].ravel()
    harvest_vec = harvest_by_tick[tick-2]       # length 400

    
    # → do whatever with 'metrics'



In [5]:
############################################################################################
# Load network from CSV file - Version 2 for Pearson weight–harvest correlations
############################################################################################

import networkx as nx
import re, pandas as pd

spec_re = re.compile(r"species(\d+)-species(\d+)")

def load_network_from_csv(fp: str) -> nx.Graph:
    df = pd.read_csv(fp)                      # the edge list file
    G  = nx.DiGraph()

    for _, row in df.iterrows():
        u, v, w = int(row['source']), int(row['target']), row['weight']
        G.add_edge(u, v, weight=w)

        # --- add / check species attribute ----------------------
        m = spec_re.fullmatch(row['type'])
        if m:
            su, sv = int(m.group(1)), int(m.group(2))
            if 'species' not in G.nodes[u]:
                G.nodes[u]['species'] = su
            if 'species' not in G.nodes[v]:
                G.nodes[v]['species'] = sv
        else:
            raise ValueError(f"type column not parsable: {row['type']}")
    return G


In [ ]:
############################################################################################
# Load network from CSV file - Version 1
############################################################################################
import networkx as nx
import pandas as pd
def load_network_from_csv(file_path):
    """
    Load network data from CSV file into a NetworkX graph
    """
    print(f"Loading network from: {file_path}")
    # Read the CSV file
    df = pd.read_csv(file_path)
    print(f"Loaded data with {len(df)} rows")
    
    # Display first few rows to verify structure
    print("\nFirst few rows of data:")
    print(df.head())
    
    # Create an empty directed graph
    G = nx.DiGraph()
    
    # Add nodes and edges to the graph
    for _, row in df.iterrows():
        source = row['source']
        target = row['target']
        weight = row['weight']
        
        # Add nodes if they don't exist
        if not G.has_node(source):
            G.add_node(source)
        if not G.has_node(target):
            G.add_node(target)
        
        # Add edge with weight
        G.add_edge(source, target, weight=weight)
    
    print(f"Created graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G


In [6]:
############################################################################################
#  Kullback-Leibler divergence 
#  This function computes K-L between the distributions of link weights and of harvests
############################################################################################

from scipy.special import rel_entr    

def kl_weight_vs_harvest(weight_vec, harvest_vec, bins='fd', eps=1e-12):
    """
    KL( P_weights || Q_harvest )  in bits.
    bins – int or 'fd'/'auto'; 40 gives finer resolution than 20.
    """
    edges = np.histogram_bin_edges(np.concatenate([weight_vec,harvest_vec]),
                               bins='fd')
    if len(edges) < 51:
        edges = np.linspace(weight_vec.min(), weight_vec.max(), 51)

    pw, _ = np.histogram(weight_vec,  bins=edges); pw = pw / pw.sum()
    qh, _ = np.histogram(harvest_vec, bins=edges); qh = qh / qh.sum()
    mask  = (pw > 0) & (qh > 0)
    if not mask.any():
        return np.nan
    kl = np.sum(pw[mask] * np.log2(pw[mask] / (qh[mask] + eps)))

    if tick in {26,78}:                # choose any harvest-aligned tick
        print("--- diagnostic tick", tick)
        print("bin edges first 6:", edges[:6])
        print("pw non-zero bins:", np.nonzero(pw>0)[0][:10])
        print("qh non-zero bins:", np.nonzero(qh>0)[0][:10])

    return max(0.0, kl)                # clip tiny negative round-off


In [14]:
###########################################################################################
# Jensen–Shannon distance
###########################################################################################

from scipy.spatial.distance import jensenshannon

def js_weight_vs_harvest(weight_vec, harvest_vec, bins='fd', eps=1e-12):
    edges = np.histogram_bin_edges(np.concatenate([weight_vec, harvest_vec]),
                                   bins=bins)
    pw,_ = np.histogram(weight_vec,  bins=edges); pw = pw / pw.sum()
    qh,_ = np.histogram(harvest_vec, bins=edges); qh = qh / qh.sum()
    return jensenshannon(pw, qh, base=2.0)   # bits



In [17]:
###########################################################################################
# CI for Jensen–Shannon distance
###########################################################################################
import numpy as np
def bootstrap_js(weight_vec, harvest_vec, B=300, bins='fd', max_n=None, seed=0):
    """
    Returns mean, 2.5% and 97.5% quantiles of JS over B bootstrap resamples.
    Uses the same preprocessing as your js_weight_vs_harvest().
    """
    rng = np.random.default_rng(seed)

    # optional downsample to balance sizes (recommended)
    if max_n is None:
        n = min(len(weight_vec), len(harvest_vec))
    else:
        n = min(max_n, len(weight_vec), len(harvest_vec))
    if n < 10:
        return np.nan, np.nan, np.nan

    js_vals = []
    for _ in range(B):
        ws = weight_vec[rng.integers(0, len(weight_vec), n)]
        hs = harvest_vec[rng.integers(0, len(harvest_vec), n)]
        js_vals.append(js_weight_vs_harvest(ws, hs, bins=bins))
    js_vals = np.array(js_vals)
    return float(np.nanmean(js_vals)), float(np.nanpercentile(js_vals, 2.5)), float(np.nanpercentile(js_vals, 97.5))


In [23]:
############################################################################################
# Append a single row to CSV, writing header if file doesn't exist/empty
############################################################################################
import os, csv

def append_csv_row(path, columns, values):
    """Append a single row to CSV, writing header if file doesn't exist/empty."""
    write_header = (not os.path.exists(path)) or (os.path.getsize(path) == 0)
    with open(path, 'a', newline='') as f:
        w = csv.writer(f)
        if write_header:
            w.writerow(columns)
        w.writerow(values)


In [ ]:
############################################################################################
# Compute structural complexity metrics for the graph
############################################################################################

import numpy as np
import networkx as nx
import os, glob
import pandas as pd
from scipy.special import rel_entr  # for relative entropy calculations
import matplotlib.pyplot as plt
from networkx.algorithms import community

def compute_structural_complexity(G, harvest_vec, tick, save_dir=None):
    """
    G               : NetworkX graph for weight tick `tick`
    harvest_vec     : np.array of 400 harvest values for tick-2    
    Compute a set of network‐structural complexity measures:
      - degree_entropy: Shannon entropy of the degree distribution (in bits)
      - weighted_degree_entropy: entropy of the strength (weighted degree) distribution
      - num_communities: number of modules found by greedy modularity
      - community_entropy: entropy of the community‐size distribution
      - modularity: the partition’s modularity score
    """
    metrics = {}
    # 1) Degree‐distribution entropy
    degs = np.array([d for _, d in G.degree()]).astype(int)
    counts = np.bincount(degs)
    probs  = counts[counts > 0] / counts.sum()
    metrics['degree_entropy'] = -np.sum(probs * np.log2(probs))
    print('degree_entropy,', metrics['degree_entropy'])
    with open('degree_entropy_results.csv', 'a') as f:
        f.write(f'{tick},{metrics["degree_entropy"]}\n')

   # 2) Strength (weighted degree) entropy – bin continuous strengths into 20 bins
    strengths = np.array([d for _, d in G.degree(weight='weight')], dtype=float)
    bins      = np.histogram_bin_edges(strengths, bins='fd')  # 'fd' for Freedman-Diaconis rule
    cW, _     = np.histogram(strengths, bins=bins)
    pW        = cW[cW > 0] / cW.sum()
    metrics['weighted_degree_entropy'] = -np.sum(pW * np.log2(pW))
    print('weighted_degree_entropy,', metrics['weighted_degree_entropy'])
    with open('weighted_degree_entropy_results.csv', 'a') as f:
        f.write(f'{tick},{metrics["weighted_degree_entropy"]}\n') 

    # 3) Kullback-Leibler divergence and Jensen–Shannon between weights & harvest
    if harvest_vec.size:
        weights_flat = np.array([d['weight'] for _,_,d in G.edges(data=True)],
                            dtype=float)

    # --- separate standardisation (shape only) -----------------
    if weights_flat.std(ddof=0) == 0 or harvest_vec.std(ddof=0) == 0:
    # one of the vectors is constant → divergences undefined
        kl_val = np.nan
        js_val = np.nan
    else:
        weights_flat = (weights_flat - weights_flat.mean()) / weights_flat.std(ddof=0)
        harvest_vec  = (harvest_vec  - harvest_vec.mean())  / harvest_vec.std(ddof=0)

    kl_val = max(0.0, kl_weight_vs_harvest(weights_flat, harvest_vec))
    js_val = js_weight_vs_harvest(weights_flat, harvest_vec)

    # optional CI (cheap: B=200 and max_n=400)
    js_mean, js_lo, js_hi = bootstrap_js(weights_flat, harvest_vec, B=200, bins='fd', max_n=400, seed=42)
    metrics['js_boot_mean'] = js_mean
    metrics['js_boot_lo']   = js_lo
    metrics['js_boot_hi']   = js_hi
    # store the results
    print('kl_weight_vs_harvest,', kl_val)
    metrics['kl_weight_vs_harvest'] = kl_val
    metrics['js_weight_vs_harvest'] = js_val

    if save_dir:
        append_csv_row(os.path.join(save_dir, 'kl_weight_vs_harvest_results.csv'),
                    ['tick','kl'], [tick, kl_val])
        append_csv_row(os.path.join(save_dir, 'js_weight_vs_harvest_results.csv'),
                    ['tick','js'], [tick, js_val])

    # NEW: CI table alongside JS
        append_csv_row(os.path.join(save_dir, 'js_with_ci_results.csv'),
                    ['tick','js','js_lo','js_hi','js_boot_mean'],
                    [tick, js_val, js_lo, js_hi, js_mean])
    else:
        # no harvest → record NaNs so downstream tables stay aligned
        metrics['kl_weight_vs_harvest'] = np.nan
        metrics['js_weight_vs_harvest'] = np.nan
        metrics['js_boot_mean'] = np.nan
        metrics['js_boot_lo']   = np.nan
        metrics['js_boot_hi']   = np.nan

    if save_dir:
        append_csv_row(os.path.join(save_dir, 'kl_weight_vs_harvest_results.csv'),
                       ['tick','kl'], [tick, ''])
        append_csv_row(os.path.join(save_dir, 'js_weight_vs_harvest_results.csv'),
                       ['tick','js'], [tick, ''])
        append_csv_row(os.path.join(save_dir, 'js_with_ci_results.csv'),
                       ['tick','js','js_lo','js_hi','js_boot_mean'],
                       [tick, '', '', '', ''])

    # 3.2)  entropy of the harvest distribution --------------
    if harvest_vec.size > 0:
        # use same bin rule as KL so results are comparable
        edges = np.histogram_bin_edges(harvest_vec, bins='fd')
        ch, _ = np.histogram(harvest_vec, bins=edges)
        ph     = ch[ch > 0] / ch.sum()
        metrics['harvest_entropy'] = -np.sum(ph * np.log2(ph))
    else:
        metrics['harvest_entropy'] = np.nan
    print('harvest_entropy,', metrics['harvest_entropy'])
    with open('harvest_entropy_results.csv', 'a') as f:
        f.write(f'{tick},{metrics["harvest_entropy"]}\n')      
   
   # 4) Community measures, detection via greedy modularity
    #    Work on undirected, weighted graph  
    G_undir    = G.to_undirected()
    comms      = list(community.greedy_modularity_communities(G_undir, weight='weight'))
    sizes      = np.array([len(c) for c in comms])
    probs_c    = sizes / sizes.sum()
    metrics['num_communities']   = len(comms)
    metrics['community_entropy'] = -np.sum(probs_c * np.log2(probs_c))
    metrics['modularity']        = community.modularity(G_undir, comms, weight='weight')
    print('community_entropy,', metrics['community_entropy'])
    with open('community_entropy_results.csv', 'a') as f:
        f.write(f'{tick},{metrics["community_entropy"]}\n')
    with open('modularity_results.csv', 'a') as f:
        f.write(f'{tick},{metrics["modularity"]}\n')

    # 5) Save the metrics to a CSV file
    return metrics



In [ ]:
#############################################################################################
# Single network analysis
# Load the network from the specified CSV file
#############################################################################################

G = load_network_from_csv(file_path)
structural = compute_structural_complexity(G)
print(structural)

In [ ]:
############################################################################################
### Simulation series network analysis
### Create a Data Frame for a time series of structural metrics
############################################################################################
  
def structural_time_series(graphs, harvest_list, ticks, save_dir=None):
    """
    Given a list of (tick, G) pairs, compute struct metrics and plot
    their evolution over time.
    Return a DataFrame of structural metrics and (optionally) save the plot.
    """   
    records = []
    for G, hv, t in zip(graphs, harvest_list, ticks):
        m = compute_structural_complexity(G, hv, t, save_dir)
        m['tick'] = t
        records.append(m)
    # ---- build DataFrame from records -------------------
    df = pd.DataFrame(records).set_index('tick').sort_index()
    print("DEBUG columns:", list(df.columns))      # <— add

    # ---- save metrics to CSV file -----------------------------
    if save_dir is not None:
        csv_name = f"structural_metrics_{ticks[0]}_{ticks[-1]}.csv"
        df.to_csv(os.path.join(save_dir, csv_name))
        print("table saved →", os.path.join(save_dir, csv_name))
    # ---- plot metrics evolution -----------------------------
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex=True)
    df['degree_entropy'        ].plot(ax=axes[0,0], marker='o')
    df['weighted_degree_entropy'].plot(ax=axes[0,1], marker='o', color='C1')
    df['community_entropy'     ].plot(ax=axes[1,0], marker='o', color='C2')
    df['num_communities'       ].plot(ax=axes[1,1], marker='o', color='C3')

    axes[0,0].set_title("Degree entropy")
    axes[0,1].set_title("Weighted-degree entropy")
    axes[1,0].set_title("Community-size entropy")
    axes[1,1].set_title("Number of communities")

    for ax in axes.flatten():
        ax.set_xlabel("Tick")
        ax.grid(alpha=0.3)
    plt.tight_layout()

    # ---- save if requested -------------------------------------
    if save_dir is not None:
        fname = f"structural_metrics_{ticks[0]}_{ticks[-1]}.png"
        fig.savefig(os.path.join(save_dir, fname), dpi=300)
        print("plot saved →", os.path.join(save_dir, fname))

    # ---- save KL divergence table and plot ---------------------
    if save_dir is not None:
        # ---- CSV table
        kl_csv = os.path.join(save_dir, 'kl_divergence_table.csv')
        df[['kl_weight_vs_harvest']].dropna().to_csv(kl_csv)
        print("KL table saved →", kl_csv)
        
    # ---- save JS divergence table and plot ---------------------
    if save_dir is not None:
        # ---- CSV table
        if 'js_weight_vs_harvest' in df.columns:
            js_csv = os.path.join(save_dir, 'js_divergence_table.csv')
            df[['js_weight_vs_harvest']].dropna().to_csv(js_csv)
            ...
        else:
            print("JS column missing — skipping JS table.")

 # ---- save the harvest entropy---------------------

    if save_dir is not None:
        # ---- CSV table
        he_csv = os.path.join(save_dir, 'harvest_entropy_results.csv')
        df[['harvest_entropy']].dropna().to_csv(he_csv, index=True)
        print("harvest_entropy table saved →", he_csv)
   
# ---- return the DataFrame with all metrics ----------------
    return df

In [ ]:
############################################################################################
### Network Analysis of structural metrics for a Simulation time series 
### This is the main part of the script that processes multiple CSV files
############################################################################################

# 1) assume `files` is your sorted list of CSV paths
# 2) define a small helper to pull the tick number out of the filename
def extract_tick(fp):
    # adjust this if your filenames differ
    return int(os.path.basename(fp).split('_')[-1].split('.')[0])

# 3) build the parallel lists

graphs       = []
ticks        = []
harvest_list = []
species_corrs = []

for fp in files:
    tick = extract_tick(fp)
    G    = load_network_from_csv(fp)
    for n in G.nodes:
        if 'species' not in G.nodes[n]:
            # infer from something, or mark as unknown
            G.nodes[n]['species'] = -1        # or whatever default you prefer

   # ------------------------------------------------------------
    # 1. harvest rows for this event (tick-2)
    # ------------------------------------------------------------
    hdf = harvest_df[harvest_df['tick'] == tick - 2]
    if hdf.empty:
        continue                           # skip non-harvest ticks

    harvest_vec = hdf['harvested'].to_numpy(float)
    # ---------- dominant species for each ilex ------------------
    tree_cols = [f'species{sp}_neighbors' for sp in range(1, 10)]
    hdf['dominant_species'] = (
        hdf[tree_cols].idxmax(axis=1).str.extract(r'(\d+)').astype(int)
    )
    # ------------------------------------------------------------
    # 2. determine dominant neighbour species for each ilex plant
    #    (insert these three lines here)
    # ------------------------------------------------------------
    tree_cols = [f'species{sp}_neighbors' for sp in range(1, 10)]
    hdf['dominant_species'] = (
        hdf[tree_cols].idxmax(axis=1)          # → "speciesX_neighbors"
           .str.extract(r'(\d+)').astype(int)  # keep just the X (1–9)
    )
    # ------------------------------------------------------------
    # 3. per-species aggregates and Pearson correlation
    # ------------------------------------------------------------

    # ---------- per-species means --------------------------------
    wbar_tick, hbar_tick = [], []
    for sp in range(1, 10):
        # --- harvest selection first ---------------------------------
        h_sel = hdf[hdf['dominant_species'] == sp]['harvested']

        # --- edge-weight selection -----------------------------------      
        w_edges = [
            d['weight']
            for u, v, d in G.edges(data=True)
            if (G.nodes[u]['species'], G.nodes[v]['species']) in {(sp, 0), (0, sp)}
        ]

        # If this species has < 3 ilex samples OR no edges, record NaN
        if (len(h_sel) < 1) or (not w_edges):
            wbar_tick.append(np.nan)
            hbar_tick.append(np.nan)
            continue

        # otherwise store the means
        wbar_tick.append(np.mean(w_edges))
        hbar_tick.append(h_sel.mean())
    # ------------------------------------------------------------
        # wbar_tick, hbar_tick grow with NaNs when no data
        # ...
        w_arr = np.array(wbar_tick, dtype=float)
        h_arr = np.array(hbar_tick, dtype=float)
        mask  = np.isfinite(w_arr) & np.isfinite(h_arr)

        if mask.sum() > 1:
            r, _ = stats.pearsonr(w_arr[mask], h_arr[mask])
        else:
            r = np.nan

    species_corrs.append({'tick': tick, 'pearson_w_h': r})

    # ---- store for the rest of your pipeline -------------------
    graphs.append(G)
    ticks.append(tick)
    harvest_list.append(harvest_vec)
          
# ------------------------------------------------------------------
#  save Pearson weight–harvest correlations per harvest tick
# ------------------------------------------------------------------
df_ts       = structural_time_series(graphs, harvest_list, ticks,
                                     save_dir=output_dir)

df_species  = (pd.DataFrame(species_corrs)
                 .set_index('tick')
                 .sort_index())
print(df_species)

print("\nNon-NaN Pearson counts:",
      df_species['pearson_w_h'].notna().sum())

# optional CSV for correlation curve
df_species.to_csv(os.path.join(output_dir,
                               "pearson_weight_vs_harvest.csv"))

# merge for plotting / joint export
df_ts = df_ts.merge(df_species, left_index=True, right_index=True, how='left')

# ---- 1. Entropy table --------------------------------------------------

entropy_table = df_ts[['weighted_degree_entropy']]          # 1-column DataFrame
entropy_csv   = os.path.join(output_dir, 'weighted_degree_entropy.csv')
entropy_table.to_csv(entropy_csv)                  # tick,weighted_degree_entropy
print("\nENTROPY TABLE\n", entropy_table.head())

# ---- 2. KL-divergence and JS per plant tables -------------------------------------------

kl_table = df_ts[['kl_weight_vs_harvest']].dropna()
kl_csv   = os.path.join(output_dir, 'kl_divergence_table.csv')
kl_table.to_csv(kl_csv)
print("\nKL TABLE\n", kl_table.head())

js_table = df_ts[['js_weight_vs_harvest']].dropna()

if 'js_weight_vs_harvest' in df_ts.columns:
    js_csv = os.path.join(save_dir, 'js_divergence_table.csv')
    df_ts[['js_weight_vs_harvest']].dropna().to_csv(js_csv)
    js_csv   = os.path.join(output_dir, 'js_divergence_table.csv')
    js_table.to_csv(js_csv)
    print("\nJS TABLE\n", js_table.head())
else:
    print("JS column missing — skipping JS table.")

#---- 3. Save all metrics to a CSV file -------------------------------
df_ts.to_csv(os.path.join(output_dir, 'all_metrics.csv'))
print("\nALL METRICS TABLE\n", df_ts.head())

# ---- 4. Plotting the time series of structural metrics ----------------
# ---- stand-alone figures

plt.figure(figsize=(6, 4))
df_ts['weighted_degree_entropy'].dropna().plot(marker='o')
plt.title("weighted_degree_entropy vs. tick")
plt.xlabel("Tick")
plt.ylabel("weighted_degree_entropy  [bits]")
plt.grid(alpha=0.3)

plt.figure(figsize=(6, 4))
df_ts['community_entropy'].dropna().plot(marker='o')
plt.title("community_entropy vs. tick")
plt.xlabel("Tick")
plt.ylabel("community_entropy   [bits]")
plt.grid(alpha=0.3)

plt.figure(figsize=(6, 4))
df_ts['num_communities'].dropna().plot(marker='o')
plt.title("num_communities vs. tick")
plt.xlabel("Tick")
plt.ylabel("num_communities  [bits]")
plt.grid(alpha=0.3)

plt.figure(figsize=(6,4))
df_ts['kl_weight_vs_harvest'].dropna().plot(marker='o')
plt.title("KL(weight‖harvest) vs. tick"); plt.xlabel("Tick"); plt.ylabel("bits")
plt.grid(alpha=0.3)
kl_png = os.path.join(output_dir,'kl_timeseries.png')
plt.savefig(kl_png,dpi=300); print("KL plot saved →", kl_png)

plt.figure(figsize=(6,4))
df_ts['js_weight_vs_harvest'].dropna().plot(marker='o')
plt.title("JS(weight‖harvest) vs. tick"); plt.xlabel("Tick"); plt.ylabel("bits")
plt.grid(alpha=0.3)
js_png = os.path.join(output_dir,'js_timeseries.png')
plt.savefig(js_png,dpi=300); print("JS plot saved →", js_png)

plt.figure(figsize=(6, 4))
df_ts['harvest_entropy'].dropna().plot(marker='o')
plt.title("Harvest_Entropy vs. tick")
plt.xlabel("Tick"); plt.ylabel("HE  [bits]")
plt.grid(alpha=0.3)
he_png = os.path.join(save_dir, 'harvest_entropy.png')
plt.savefig(he_png, dpi=300)
print("HE plot  saved →", he_png)

plt.figure(figsize=(6,4))
df_ts['pearson_w_h'].dropna().plot(marker='o')
plt.title("Pearson (weight mean  vs  harvest mean) over time")
plt.xlabel("Tick"); plt.ylabel("r")
plt.grid(alpha=0.3)


plt.tight_layout()
plt.show()

# Save tidy JS+CI table
if {'js_weight_vs_harvest','js_boot_lo','js_boot_hi'} <= set(df_ts.columns):
    js_ci = df_ts[['js_weight_vs_harvest','js_boot_lo','js_boot_hi']].dropna()
    js_ci.to_csv(os.path.join(output_dir, 'js_timeseries_with_ci.csv'))

    # Standalone figure with CI ribbon
    plt.figure(figsize=(6,4))
    x = js_ci.index.values
    plt.plot(x, js_ci['js_weight_vs_harvest'], marker='o', label='JS')
    plt.fill_between(x, js_ci['js_boot_lo'], js_ci['js_boot_hi'], alpha=0.2, label='95% CI')
    plt.title('JS(weight‖harvest) with bootstrap CI')
    plt.xlabel('Tick'); plt.ylabel('JS [bits]'); plt.grid(alpha=0.3); plt.legend()
    out_png = os.path.join(output_dir, 'js_timeseries_with_ci.png')
    plt.savefig(out_png, dpi=300, bbox_inches='tight')
    print('JS+CI plot saved →', out_png)



In [58]:
print(df_species['pearson_w_h'].isna().value_counts())


pearson_w_h
False    9
True     1
Name: count, dtype: int64


In [ ]:
############################################################################################
# --- DIAGNOSE quick post-run diagnosis -------------------
############################################################################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

TEST_TICK = 338                # <- put any weight tick that has harvest (26,78,…)

# pull vectors that went into the metric
G_test        = graphs[ticks.index(TEST_TICK)]
weights_test  = np.array([d['weight'] for _,_,d in G_test.edges(data=True)])
harvest_test  = harvest_df.loc[harvest_df['tick']==TEST_TICK-2,'harvested'].to_numpy(float)

# same bin rule you used in the metric
edges = np.histogram_bin_edges(np.concatenate([weights_test,harvest_test]), bins='fd')
pw,_ = np.histogram(weights_test,  bins=edges); pw = pw/pw.sum()
qh,_ = np.histogram(harvest_test,  bins=edges);  qh = qh/qh.sum()

print(f"\nDEBUG  tick {TEST_TICK} (weights)  & {TEST_TICK-2} (harvest)")
print("edges min..max:", edges[0], edges[-1], "   bins:", len(edges)-1)
print("weight mass in bins:", np.nonzero(pw>0)[0][:10], " …")
print("harvest mass bins:",    np.nonzero(qh>0)[0][:10], " …")

# show both histograms
fig,ax = plt.subplots(figsize=(6,3))
wcent = 0.5*(edges[:-1]+edges[1:])
ax.step(wcent, pw, where='mid', label='weights', linewidth=1.5)
ax.step(wcent, qh, where='mid', label='harvest', linewidth=1.5)
ax.legend(); ax.set(title=f"Tick {TEST_TICK}: histogram overlay", ylabel="prob.");
plt.show()


In [ ]:
############################################################################################
# --- DIAGNOSE choose a harvest-aligned tick to inspect -------------
############################################################################################
TEST_TICK = 338                                    # 338 → harvest 336
G_test     = graphs[ticks.index(TEST_TICK)]
w_raw      = np.array([d['weight'] for _,_,d in G_test.edges(data=True)],
                      dtype=float)
h_raw      = harvest_df.loc[harvest_df['tick']==TEST_TICK-2,'harvested'
                           ].to_numpy(float)

# ---- separate standardisation, just like in the metric ------------
if w_raw.std(ddof=0) > 0 and h_raw.std(ddof=0) > 0:
    w = (w_raw - w_raw.mean()) / w_raw.std(ddof=0)
    h = (h_raw - h_raw.mean()) / h_raw.std(ddof=0)
else:
    raise ValueError("One vector is constant at this tick")

edges = np.histogram_bin_edges(np.concatenate([w, h]), bins='fd')
pw,_  = np.histogram(w, bins=edges); pw = pw / pw.sum()
qh,_  = np.histogram(h, bins=edges); qh = qh / qh.sum()

print(f"DEBUG  tick {TEST_TICK}:")
print("edges min..max:", edges[0], edges[-1], " bins:", len(edges)-1)
print("w bins with mass:", np.nonzero(pw>0)[0][:10])
print("h bins with mass:", np.nonzero(qh>0)[0][:10])

import matplotlib.pyplot as plt
cent = 0.5*(edges[:-1]+edges[1:])
plt.step(cent, pw, where='mid', label='weights (z)', linewidth=1.4)
plt.step(cent, qh, where='mid', label='harvest (z)', linewidth=1.4)
plt.legend(); plt.title(f"Histogram overlay (z-score) – tick {TEST_TICK}")
plt.show()


In [ ]:
# ---------------------------------------------------------------
# Full-run diagnostic: overlay histograms for *every* harvest tick
# ---------------------------------------------------------------
import matplotlib.pyplot as plt
import numpy as np
import os

PLOT_EACH   = True                  # True → pop a window for every tick
SAVE_PNGS   = False                   # True → save PNGs in diag_dir
diag_dir    = os.path.join(output_dir, "diagnostic_plots")
os.makedirs(diag_dir, exist_ok=True)

for fp in files:                                         # your sorted CSV list
    tick = extract_tick(fp)                              # 0, 26, 52, …
    hdf  = harvest_df[harvest_df["tick"] == tick - 2]    # harvest tick-2
    if hdf.empty:
        continue                                         # skip non-harvest ticks

    # --- raw vectors ----------------------------------------------
    G_test = load_network_from_csv(fp)
    w_raw  = np.array([d['weight'] for _,_,d in G_test.edges(data=True)],
                      dtype=float)
    h_raw  = hdf['harvested'].to_numpy(float)

    if w_raw.std(ddof=0) == 0 or h_raw.std(ddof=0) == 0:
        print(f"tick {tick}: one vector is constant; skip")
        continue

    # --- same z-score as in metric -------------------------------
    w = (w_raw - w_raw.mean()) / w_raw.std(ddof=0)
    h = (h_raw - h_raw.mean()) / h_raw.std(ddof=0)

    edges = np.histogram_bin_edges(np.concatenate([w, h]), bins='fd')
    pw,_ = np.histogram(w, bins=edges); pw = pw / pw.sum()
    qh,_ = np.histogram(h, bins=edges); qh = qh / qh.sum()

    # quick text summary
    print(f"tick {tick:3d}  bins={len(edges)-1:3d}  overlap bins="
          f"{np.sum((pw>0)&(qh>0))}")

    # optional figure
    if PLOT_EACH or SAVE_PNGS:
        cent = 0.5*(edges[:-1]+edges[1:])
        plt.figure(figsize=(5,3))
        plt.step(cent, pw, where='mid', label='weights (z)')
        plt.step(cent, qh, where='mid', label='harvest (z)')
        plt.title(f"Histogram overlay – weights vs harvest  (tick {tick})")
        plt.xlabel("z-score"); plt.ylabel("probability")
        plt.legend(); plt.tight_layout()
        if SAVE_PNGS:
            fpng = os.path.join(diag_dir, f"overlay_tick_{tick}.png")
            plt.savefig(fpng, dpi=300)
        if PLOT_EACH:
            plt.show()
        else:
            plt.close()

print("\nDiagnostics complete.  PNGs (if saved) are in:", diag_dir)


In [115]:
############################################################################################
# Visualize structural metrics for a single graph
############################################################################################

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def visualize_structural_one(G, ax_deg=None, ax_str=None, ax_comm=None):
    """
    Given a single graph G, plot:
      - unweighted degree hist
      - weighted‐degree (strength) hist
      - community‐size bar chart
    Returns the three Axes so you can plt.tight_layout() on the figure.
    """
    # 1) Degree histogram
    degs = np.array([d for _, d in G.degree()])
    if ax_deg is None:
        fig = plt.figure(figsize=(12, 8))
        gs  = fig.add_gridspec(2, 2)
        ax_deg  = fig.add_subplot(gs[0, 0])
        ax_str  = fig.add_subplot(gs[0, 1])
        ax_comm = fig.add_subplot(gs[1, :])
    ax_deg.hist(degs, bins=range(degs.min(), degs.max()+2), edgecolor='black', alpha=0.7)
    ax_deg.set_title("Degree distribution")
    ax_deg.set_xlabel("Degree")
    ax_deg.set_ylabel("Count")

    # 2) Strength histogram
    strengths = np.array([d for _, d in G.degree(weight='weight')])
    ax_str.hist(strengths, bins=20, edgecolor='black', alpha=0.7)
    ax_str.set_title("Strength (weighted degree) distribution")
    ax_str.set_xlabel("Strength")
    ax_str.set_ylabel("Count")

    # 3) Community sizes
    G_undir    = G.to_undirected()
    communities = list(community.greedy_modularity_communities(G_undir, weight='weight'))
    sizes      = [len(c) for c in communities]
    ax_comm.bar(range(len(sizes)), sorted(sizes, reverse=True), color='C2', alpha=0.8)
    ax_comm.set_title("Community sizes (greedy modularity)")
    ax_comm.set_xlabel("Community rank")
    ax_comm.set_ylabel("Size (# nodes)")

    return ax_deg, ax_str, ax_comm





In [116]:
############################################################################################
# Visualize structural metrics for a time series of graphs
# This function takes a list of graphs and their corresponding ticks
############################################################################################
def structural_time_series(graphs, ticks):
    """
    Given a list of (tick, G) pairs, compute struct metrics and plot
    their evolution over time.
    """
    records = []
    for tick, G in zip(ticks, graphs):
        m = compute_structural_complexity(G)
        m['tick'] = tick
        records.append(m)
    df = pd.DataFrame(records).set_index('tick').sort_index()

    # plot time series
    fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex=True)
    df['degree_entropy'].plot(ax=axes[0,0], marker='o')
    axes[0,0].set_title("Degree-entropy over time")
    df['weighted_degree_entropy'].plot(ax=axes[0,1], marker='o', color='C1')
    axes[0,1].set_title("Weighted-degree entropy over time")
    df['community_entropy'].plot(ax=axes[1,0], marker='o', color='C2')
    axes[1,0].set_title("Community-size entropy over time")
    df['num_communities'].plot(ax=axes[1,1], marker='o', color='C3')
    axes[1,1].set_title("Number of communities over time")

    for ax in axes.flatten():
        ax.set_xlabel("Tick")
        ax.grid(alpha=0.3)
    plt.tight_layout()
    return df



In [ ]:
# -----------------------
# Example usage for a single G at tick t:
# -----------------------
for tick, G in zip(ticks, graphs):
    print(f"Visualizing structural metrics for tick {tick}")
ax1, ax2, ax3 = visualize_structural_one(G)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------
# Example usage across many ticks:
# -----------------------
# suppose you have
#    graphs = [load_network(fp) for fp in files]
#    ticks  = [extract_tick(fp) for fp in files]
df_ts = structural_time_series(graphs, harvest_list, ticks, save_dir=output_dir)
plt.show()